In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta
from tqdm import tqdm
import gc
import sys
import matplotlib.pyplot as plt
import MetaTrader5 as mt5

path_root = "D:/Documentos/Erik/TDR/TDR-Forex/"
sys.path.append(path_root)
from utils.mt5 import *
from utils.data import get_dataset
from utils.strategies import get_strategy_H1, CalcLotSize

## Posicions

In [10]:
# Retorna una taula amb les posicions obertes
posicions = get_positions_df()
posicions

,ticket,time,time_msc,time_update,time_update_msc,type,magic,identifier,reason,volume,price_open,sl,tp,price_current,swap,profit,symbol,comment,external_id
0,4412362562,2025-08-25 11:55:40,2025-08-25 11:55:40.172,2025-08-25 11:55:40,2025-08-25 11:55:40.172,0,1974,4412362562,3,0.1,1.17070,0.00000,0.00000,1.17047,0.0,-1.97,EURUSD,Twin Hours EMA,
1,4412368652,2025-08-25 11:56:21,2025-08-25 11:56:21.508,2025-08-25 11:56:21,2025-08-25 11:56:21.508,0,1974,4412368652,3,0.1,1.17048,1.16948,1.17298,1.17047,0.0,-0.09,EURUSD,Twin Hours EMA,
2,4412371490,2025-08-25 11:56:41,2025-08-25 11:56:41.034,2025-08-25 11:56:41,2025-08-25 11:56:41.034,1,1974,4412371490,3,0.1,1.17036,1.17136,1.16786,1.17059,0.0,-1.96,EURUSD,Twin Hours EMA,


## Deals

#### `type`: Indica el tipus d’operació o transacció:
- **0**: Buy  
- **1**: Sell  
- **2**: Balance  

#### `entry`: Especifica com s’ha introduït o tancat la posició:
- **0**: In (Obertura)  
- **1**: Out (Tancament)  
- **2**: In/Out (Modificació parcial, increment o reducció)  

#### `reason`: Mostra la raó per la qual s’ha executat l’operació:
- **0**: Client (Execució manual)  
- **1**: Expert (Expert Advisor)  
- **2**: Dealer (Broker)  
- **3**: Signal (Subscripció a senyal)  
- **4**: Stop Loss (SL activat)  
- **5**: Take Profit (TP activat)  
- **6**: Stop Out (Tancament forçat per marge)  
- **7**: Roll Over (Ajustaments com swaps o contractes)  
- **8**: SL/TP (Altres execucions automàtiques)  


In [11]:
# Retorna una operacions realitzades
deals = get_deals_df()
deals.tail(2)

,ticket,order,time,time_msc,type,entry,magic,position_id,reason,volume,price,commission,swap,profit,fee,symbol,comment,external_id
68,4358368166,4412368652,2025-08-25 11:56:21,2025-08-25 11:56:21.508,0,0,1974,4412368652,3,0.1,1.17048,0.0,0.0,0.0,0.0,EURUSD,Twin Hours EMA,
69,4358371111,4412371490,2025-08-25 11:56:41,2025-08-25 11:56:41.034,1,0,1974,4412371490,3,0.1,1.17036,0.0,0.0,0.0,0.0,EURUSD,Twin Hours EMA,


### Obrir i modificar operacions

In [7]:
# Obrir una posició llarga. Retorna l'identificador de l'operació
argSymbol = "EURUSD"; argLotSize = 0.10; argSlippage = 10; argMagicNumber = 1974; argComment = "Twin Hours EMA"; argSLPips = 10; argTPPips =25
ticket = OpenBuyOrder(argSymbol, argLotSize, argSlippage, argMagicNumber, argComment, argSLPips, argTPPips)
ticket

Obrint ordre BUY: EURUSD 0.1 lots
Preu: 1.17048
SL: 1.16948 (10 pips)
TP: 1.17298 (25 pips)
Ordre executada correctament!
   Order: 4412368652, Deal: 4358368166
   SL: 1.16948
   TP: 1.17298


4412368652

In [8]:
# Obrir una posició curta. Retorna l'identificador de l'operació
argSymbol = "EURUSD"; argLotSize = 0.10; argSlippage = 10; argMagicNumber = 1974; argComment = "Twin Hours EMA"; argSLPips = 10; argTPPips =25
ticket = OpenSellOrder(argSymbol, argLotSize, argSlippage, argMagicNumber, argComment, argSLPips, argTPPips)
ticket

Obrint ordre SELL: EURUSD 0.1 lots
Preu: 1.17036
SL: 1.17136 (10 pips)
TP: 1.16786 (25 pips)
Ordre de venda executada correctament!
   Order: 4412371490, Deal: 4358371111
   SL: 1.17136
   TP: 1.16786


4412371490

In [9]:
posicions = get_positions_df()
posicions

,ticket,time,time_msc,time_update,time_update_msc,type,magic,identifier,reason,volume,price_open,sl,tp,price_current,swap,profit,symbol,comment,external_id
0,4412362562,2025-08-25 11:55:40,2025-08-25 11:55:40.172,2025-08-25 11:55:40,2025-08-25 11:55:40.172,0,1974,4412362562,3,0.1,1.17070,0.00000,0.00000,1.17035,0.0,-2.99,EURUSD,Twin Hours EMA,
1,4412368652,2025-08-25 11:56:21,2025-08-25 11:56:21.508,2025-08-25 11:56:21,2025-08-25 11:56:21.508,0,1974,4412368652,3,0.1,1.17048,1.16948,1.17298,1.17035,0.0,-1.11,EURUSD,Twin Hours EMA,
2,4412371490,2025-08-25 11:56:41,2025-08-25 11:56:41.034,2025-08-25 11:56:41,2025-08-25 11:56:41.034,1,1974,4412371490,3,0.1,1.17036,1.17136,1.16786,1.17048,0.0,-1.03,EURUSD,Twin Hours EMA,


In [15]:
# Afegim SL i TP a una posició oberta
Modify_SL_and_TP(ticket, 1.18, 1.16)

Change SL-TP 4412371490 successfully


In [16]:
# Tanquem una posició pel seu indicador
CloseOrderByTicket(ticket, 10)

Ticket 4412371490 closed successfully


In [17]:
# Tanquem totes les posicions obertes d'una divisa i un identificador
CloseAllOrders("EURUSD", 1974, 10)

Closed successfully: retcode=10009 - deal=4358444823 - order=4412445184
Closed successfully: retcode=10009 - deal=4358444828 - order=4412445189


In [19]:
# Creem una ordre pendent
order = fct_mt5.OpenPendingOrder("EURUSD", 0.01, 1.1625, 1.1575, 1.1650, 10, 2006, "Prova", "ORDER_TYPE_BUY_LIMIT")

Pending order placed successfully: retcode=10009 - deal=0 - order=4381769014
